In [1]:
from dandi.dandiapi import DandiAPIClient
from pynwb import NWBHDF5IO

with DandiAPIClient() as client:
    asset = client.get_dandiset("000231").get_asset_by_path(
        "sub-KF134/sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb"
    )
    url = asset.get_content_url(follow_redirects=1, strip_query=True)

io = NWBHDF5IO(url, mode="r", load_namespaces=True, driver="ros3")
nwb = io.read()

# What's inside?
print("Acquisition:", list(nwb.acquisition.keys()))
print("Processing:", list(nwb.processing.keys()))
print("Units:", nwb.units if nwb.units else "None")
print("Trials:", nwb.trials.to_dataframe().head() if nwb.trials else "None")
print("Electrodes:", nwb.electrodes.to_dataframe().head())

Acquisition: ['behavioral video', 'extracellular array recording']
Processing: ['behavior']
Units: units pynwb.misc.Units at 0x2514637048192
Fields:
  colnames: ['inhibitory' 'depth' 'layer' 'neuron_channel' 'spike_times'
 'obs_intervals']
  columns: (
    inhibitory <class 'hdmf.common.table.VectorData'>,
    depth <class 'hdmf.common.table.VectorData'>,
    layer <class 'hdmf.common.table.VectorData'>,
    neuron_channel <class 'hdmf.common.table.VectorData'>,
    spike_times_index <class 'hdmf.common.table.VectorIndex'>,
    spike_times <class 'hdmf.common.table.VectorData'>,
    obs_intervals_index <class 'hdmf.common.table.VectorIndex'>,
    obs_intervals <class 'hdmf.common.table.VectorData'>
  )
  description: Autogenerated by NWBFile
  id: id <class 'hdmf.common.table.ElementIdentifiers'>
  waveform_unit: volts

Trials:     start_time  stop_time  direct_delivery  stim_is_random  optogenetic  \
id                                                                        
88     399

In [3]:
from dandi.dandiapi import DandiAPIClient
from pynwb import NWBHDF5IO


with NWBHDF5IO(str(r"C:\Users\aksel\Documents\Code\EthoGraph\data\sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb"), "r") as io:
    nwb = io.read()

    # Find video references
    for name, obj in nwb.acquisition.items():
        if hasattr(obj, 'external_file'):
            print(f"{name}: {obj.external_file[:]}")
            print(f"  rate: {obj.rate}")
            print(f"  starting_time: {obj.starting_time}")

    # Find processed data (Kilosort results are here as units)
    if nwb.units:
        units_df = nwb.units.to_dataframe()
        print(f"\n{len(units_df)} sorted units (from Kilosort+Phy)")
        print(f"Columns: {units_df.columns.tolist()}")

    # Processing modules often contain behavioral or LFP data
    for mod_name, mod in nwb.processing.items():
        print(f"\nProcessing module '{mod_name}':")
        for container_name in mod.data_interfaces:
            print(f"  {container_name}: {type(mod[container_name]).__name__}")

behavioral video: ['sub-KF134_ses-20180211T154918_behavior+ecephys+image/26871407-4821-4cd0-bde7-38e14b3c13ba_external_file_0.mkv']
  rate: 200.0
  starting_time: 0.0

14 sorted units (from Kilosort+Phy)
Columns: ['inhibitory', 'depth', 'layer', 'neuron_channel', 'spike_times', 'obs_intervals']

Processing module 'behavior':
  licks: BehavioralEvents
  processed_position_whisker_C0: BehavioralTimeSeries
  processed_position_whisker_C1: BehavioralTimeSeries
  processed_position_whisker_C2: BehavioralTimeSeries
  processed_position_whisker_C3: BehavioralTimeSeries
  raw_position_whisker_C0: PoseEstimation
  raw_position_whisker_C1: PoseEstimation
  raw_position_whisker_C2: PoseEstimation
  raw_position_whisker_C3: PoseEstimation
  rewards: BehavioralEvents
  contacts_by_whisker_C0: TimeIntervals
  contacts_by_whisker_C1: TimeIntervals
  contacts_by_whisker_C2: TimeIntervals
  contacts_by_whisker_C3: TimeIntervals


In [2]:
import pynwb

with pynwb.NWBHDF5IO(r"C:\Users\aksel\Documents\Code\EthoGraph\data\sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb", "r") as io:
    nwb = io.read()
    
    # Check for video/image data
    print(nwb.acquisition)
    
    # Look for ImageSeries
    for name, obj in nwb.acquisition.items():
        print(f"{name}: {type(obj).__name__}, shape: {getattr(obj, 'data', None).shape if hasattr(obj, 'data') else 'N/A'}")

{'behavioral video': behavioral video pynwb.image.ImageSeries at 0x2111836426160
Fields:
  comments: All times in the NWB file are defined with respect to this video, which starts at 0 and proceeds at exactly 200 fps
  conversion: 1.0
  data: <HDF5 dataset "data": shape (0, 0, 0), type "|u1">
  description: External file: behavioral video of whiskers
  device: high-speed camera pynwb.device.Device at 0x2111832668256
Fields:
  description: DR1-D1312IE-100-G2-8
  manufacturer: Photonfocus

  dimension: <HDF5 dataset "dimension": shape (2,), type "<i8">
  external_file: <StrDataset for HDF5 dataset "external_file": shape (1,), type "|O">
  format: external
  offset: 0.0
  rate: 200.0
  resolution: -1.0
  starting_frame: [0]
  starting_time: 0.0
  starting_time_unit: seconds
  unit: unknown
, 'extracellular array recording': extracellular array recording pynwb.ecephys.ElectricalSeries at 0x2111839910608
Fields:
  comments: broken channels excluded. original sampling was at 30 kHz but times

In [3]:
print(nwb.acquisition['behavioral video'].external_file[:])

ValueError: Invalid dataset identifier (identifier is not of specified type)

In [5]:
import h5py

f = h5py.File(r"C:\Users\aksel\Documents\Code\EthoGraph\data\sub-KF134_ses-20180211T154918_behavior+ecephys+image.nwb", "r")

# Navigate to the external_file dataset directly
ext = f['acquisition']['behavioral video']['external_file']
print(ext)
print(ext.shape)
print(ext[()])

<HDF5 dataset "external_file": shape (1,), type "|O">
(1,)
[b'sub-KF134_ses-20180211T154918_behavior+ecephys+image/26871407-4821-4cd0-bde7-38e14b3c13ba_external_file_0.mkv']


In [4]:
from one.api import ONE
import numpy as np
import pandas as pd
from pathlib import Path

ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')

eid = 'b69b86be-af7d-4ecf-8cbf-0cd356afa1bd'

# 1. Videos (3 cameras) - these are large, ~1-5 GB each
for cam in ['left', 'right', 'body']:
    one.load_dataset(eid, f'_iblrig_{cam}Camera.raw.mp4', collection='raw_video_data', download_only=True)

# # 2. Camera timestamps + Lightning Pose
# for cam in ['left', 'right', 'body']:
#     one.load_dataset(eid, f'_ibl_{cam}Camera.times.npy', collection='alf', download_only=True)
#     one.load_dataset(eid, f'_ibl_{cam}Camera.lightningPose.pqt', collection='alf', download_only=True)

# # 3. Spike sorting (pykilosort)
# spike_times = one.load_dataset(eid, 'spikes.times.npy', collection='alf/probe00/pykilosort')
# spike_clusters = one.load_dataset(eid, 'spikes.clusters.npy', collection='alf/probe00/pykilosort')
# spike_amps = one.load_dataset(eid, 'spikes.amps.npy', collection='alf/probe00/pykilosort')
# spike_depths = one.load_dataset(eid, 'spikes.depths.npy', collection='alf/probe00/pykilosort')
# cluster_metrics = one.load_dataset(eid, 'clusters.metrics.pqt', collection='alf/probe00/pykilosort')
# cluster_channels = one.load_dataset(eid, 'clusters.channels.npy', collection='alf/probe00/pykilosort')

# # 4. Trials
# trials = one.load_object(eid, 'trials', collection='alf')
# trials_df = trials.to_df()
# print(f"Trials: {len(trials_df)}")
# print(trials_df.head())

# # 5. Slice one trial (e.g. trial 10)
# trial_idx = 10
# t_start = trials_df.iloc[trial_idx]['intervals_0']  # or 'stimOn_times'
# t_end = trials_df.iloc[trial_idx]['intervals_1']
# print(f"\nTrial {trial_idx}: {t_start:.3f}s - {t_end:.3f}s")

# # Spikes in this trial
# mask = (spike_times >= t_start) & (spike_times <= t_end)
# print(f"Spikes in trial: {mask.sum()}")
# print(f"Active clusters: {np.unique(spike_clusters[mask]).size}")

# # Video frames in this trial
# cam_times = one.load_dataset(eid, '_ibl_leftCamera.times.npy', collection='alf')
# frame_mask = (cam_times >= t_start) & (cam_times <= t_end)
# print(f"Left camera frames in trial: {frame_mask.sum()}")
# print(f"Frame indices: {np.where(frame_mask)[0][[0, -1]]}")

Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"


(S3) C:\Users\aksel\Downloads\ONE\openalyx.internationalbrainlab.org\zadorlab\Subjects\CSH_ZAD_026\2020-08-19\001\raw_video_data\_iblrig_leftCamera.raw.mp4: 100%|██████████| 3.53G/3.53G [05:13<00:00, 11.3MB/s]
(S3) C:\Users\aksel\Downloads\ONE\openalyx.internationalbrainlab.org\zadorlab\Subjects\CSH_ZAD_026\2020-08-19\001\raw_video_data\_iblrig_rightCamera.raw.mp4: 100%|██████████| 3.58G/3.58G [05:57<00:00, 10.0MB/s]
100%|██████████| 3/3.0 [00:00<00:00,  3.98it/s]


In [3]:
one.load_dataset(eid, '_iblrig_bodyCamera.raw.mp4', collection='raw_video_data', download_only=True)

(S3) C:\Users\aksel\Downloads\ONE\openalyx.internationalbrainlab.org\zadorlab\Subjects\CSH_ZAD_026\2020-08-19\001\raw_video_data\_iblrig_bodyCamera.raw.mp4: 100%|██████████| 736M/736M [01:10<00:00, 10.5MB/s] 


WindowsALFPath('C:/Users/aksel/Downloads/ONE/openalyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_026/2020-08-19/001/raw_video_data/_iblrig_bodyCamera.raw.mp4')

In [2]:
from one.api import ONE

one = ONE(base_url='https://openalyx.internationalbrainlab.org')

# Find a session with video
eid = one.search(task_protocol='_iblrig_tasks_biasedChoiceWorld', 
                 dataset_types=['_iblrig_leftCamera.raw.mp4'],
                 limit=1)[0]

# Download just the left camera video
video_path = one.load_dataset(eid, '_iblrig_leftCamera.raw.mp4')
print(video_path)

IndexError: list index out of range

In [3]:
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org')

# Search without filters first
eids = one.search(limit=5)
print(eids)

# Check what datasets a session has
if eids:
    datasets = one.list_datasets(eids[0])
    print([d for d in datasets if 'Camera' in d or 'camera' in d])

['alf/_ibl_bodyCamera.dlc.pqt', 'alf/_ibl_bodyCamera.times.npy', 'alf/_ibl_leftCamera.dlc.pqt', 'alf/_ibl_leftCamera.features.pqt', 'alf/_ibl_leftCamera.times.npy', 'alf/_ibl_rightCamera.dlc.pqt', 'alf/_ibl_rightCamera.features.pqt', 'alf/_ibl_rightCamera.times.npy', 'alf/bodyCamera.ROIMotionEnergy.npy', 'alf/leftCamera.ROIMotionEnergy.npy', 'alf/rightCamera.ROIMotionEnergy.npy', 'raw_video_data/_iblrig_bodyCamera.raw.mp4', 'raw_video_data/_iblrig_leftCamera.raw.mp4', 'raw_video_data/_iblrig_rightCamera.raw.mp4']


In [ ]:
import vocalpy as voc
from ethograph.features.energy import meansquared

sound = voc.Sound.read("path/to/audio.wav")
data = sound.data[0]  # first channel, 1-D
sr = sound.samplerate

env_time, envelope = meansquared(data, sr)

In [4]:
eid = eids[0]
video_path = one.load_dataset(eid, 'raw_video_data/_iblrig_leftCamera.raw.mp4')
print(video_path)

(S3) C:\Users\aksel\Downloads\ONE\openalyx.internationalbrainlab.org\churchlandlab_ucla\Subjects\FD_28\2023-12-01\001\raw_video_data\_iblrig_leftCamera.raw.mp4: 100%|██████████| 8.10G/8.10G [12:04<00:00, 11.2MB/s]

C:\Users\aksel\Downloads\ONE\openalyx.internationalbrainlab.org\churchlandlab_ucla\Subjects\FD_28\2023-12-01\001\raw_video_data\_iblrig_leftCamera.raw.mp4


In [ ]:
env_time, envelope = env_ava(audio_data_1d, sample_rate)

In [8]:
from dandi.dandiapi import DandiAPIClient
from pathlib import Path

dest = Path(r"C:\Users\aksel\Documents\Code\EthoGraph\data")

with DandiAPIClient() as client:
    ds = client.get_dandiset("000231")
    for asset in ds.get_assets():
        if "mkv" in asset.path:
            print(f"Downloading: {asset.path} ({asset.size / 1e6:.1f} MB)")
            asset.download(dest / Path(asset.path).name)
            print("Done")

Downloading: sub-KM91/sub-KM91_ses-20161216T143049_behavior+image/d38273f1-2e32-42e7-a8d1-ab5c7a442a7b_external_file_0.mkv (8753.2 MB)


KeyboardInterrupt: 

In [6]:
from dandi.dandiapi import DandiAPIClient

with DandiAPIClient() as client:
    ds = client.get_dandiset("000231")
    for asset in ds.get_assets():
        if "mkv" in asset.path or "video" in asset.path.lower():
            print(f"{asset.path}  ({asset.size / 1e6:.1f} MB)")
            print(f"  {asset.get_content_url(follow_redirects=1, strip_query=True)}")

sub-KM91/sub-KM91_ses-20161216T143049_behavior+image/d38273f1-2e32-42e7-a8d1-ab5c7a442a7b_external_file_0.mkv  (8753.2 MB)
  https://dandiarchive.s3.amazonaws.com/blobs/fa6/701/fa670161-4a82-44e9-b647-116c5b08397e
sub-KM91/sub-KM91_ses-20170313T135620_behavior+image/3aa9533a-58f2-49c5-98be-90ae2e542e95_external_file_0.mkv  (7456.4 MB)
  https://dandiarchive.s3.amazonaws.com/blobs/4df/41c/4df41c5b-5555-4c07-83fc-6343d52e96e2
sub-KM91/sub-KM91_ses-20170118T170550_behavior+image/109b690d-3258-4d75-a6cb-d6a8d3555ec9_external_file_0.mkv  (9912.5 MB)
  https://dandiarchive.s3.amazonaws.com/blobs/5d5/4b1/5d54b138-cc78-4eda-a709-ff530e25df6a
sub-KM91/sub-KM91_ses-20170119T182155_behavior+image/b836613c-73f6-43e6-b6c5-60177b3bd9e9_external_file_0.mkv  (10371.4 MB)
  https://dandiarchive.s3.amazonaws.com/blobs/9c8/c45/9c8c451e-1caf-471d-af04-0b0f7dee7da8
sub-KM101/sub-KM101_ses-20170714T132037_behavior+image/11b7fd02-9768-4c41-b116-b788c29cc0be_external_file_0.mkv  (5342.1 MB)
  https://dandiarc